In [7]:
from autograd import numpy as np, jacobian

In [8]:
def NewtonMethod(F: callable, 
                 x0: np.ndarray, 
                 tol: float, 
                 maxit: int
) -> np.ndarray:

    err = []

    jac = jacobian(F)
    xi = x0
    Fe = F(xi)
    cond = np.linalg.norm(F(x0)) > tol
    i = 0

    while cond and i < maxit:
          
        s = np.linalg.solve(a = jac(xi),
                               b = -Fe)
        
        err.append(np.linalg.norm(s, ord=np.inf))
        
        xi = xi + s
        i += 1
        Fe = F(xi)
        cond = np.linalg.norm(Fe) > tol
    
    return xi, err

def BroydenMethod(F: callable, 
                 x0: np.ndarray, 
                 tol: float, 
                 maxit: int
) -> np.ndarray:
    
    err = []
    J = jacobian(F)(x0)
    xi = x0
    Fe = F(xi)
    cond = np.linalg.norm(Fe) > tol
    i = 0

    while cond and i < maxit:

        s = np.linalg.solve(a = J,
                            b = -Fe)
        
        err.append(np.linalg.norm(s, ord=np.inf))

        yi = F(xi + s) - Fe
        xi = xi + s
        Fe = F(xi)
        i += 1
        cond = np.linalg.norm(Fe) > tol
        J += np.inner((yi - J @ s), s) / np.inner(s,s)
    
    return xi, err

In [9]:
def f1(x1, x2, x3):
    return 3 * x1 - np.cos(x2 * x3) - (1 / 2)

def f2(x1, x2, x3):
    return np.exp(-x1 * x2) + 20 * x3 + (10 * np.pi - 3) / 3

def f3(x1, x2, x3):
    return x1**2 - 81 * (x2 + 0.1)**2 + np.sin(x3) + 1.06

def F(x):
    return np.array([f1(x[0], x[1], x[2]),
                    f2(x[0], x[1], x[2]),
                    f3(x[0], x[1], x[2])])

x0 = np.array([0.1,0.1,-0.1])

In [21]:
xN, errN = NewtonMethod(F=F,
                  x0=x0,
                  tol=1e-32,
                  maxit=10)

xB, errB = BroydenMethod(F=F,
                  x0=x0,
                  tol=1e-32,
                  maxit=10)

Here I am coparing the convergence of each method using the infinity norm between the values the method visit between contiguous iterations

In [22]:
# Print the header
print(f"{'Iteration':<20}{'Newton error':<40}{'Broyden error':<20}")

# Print the table rows
for index, (item1, item2) in enumerate(zip(errN, errB)):
    print(f"{index:<20}{item1:<40}{item2:<20}")

Iteration           Newton error                            Broyden error       
0                   0.42152047193583064                     0.42152047193583064 
1                   0.017878257167124205                    0.010798979908119347
2                   0.0015761465869723393                   0.0046436351566131  
3                   1.244400753583079e-05                   0.002039759145408907
4                   7.757857127143586e-10                   0.0010227667732147945
5                   8.893670222225524e-17                   0.00047626294944988105
6                   8.927982221539666e-17                   0.00024944908815005453
7                   8.927982221539666e-17                   0.00011693908549938023
8                   8.927982221539666e-17                   6.101156418130812e-05
9                   8.927982221539666e-17                   2.8691764354820767e-05
